<a href="https://colab.research.google.com/github/tkhan3/machinelearning/blob/master/gensim_topic_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
#import pandas as pd
!google-drive-ocamlfuse drive
!apt-get -qq install -y graphviz && pip install -q pydot
!pip3 install seaborn==0.9.0
import pydot
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install sklearn_pandas

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131352 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import os
from os import chdir,listdir,path,getcwd
from sys import argv,exit
import pandas as pd
import re
#from mkl import *
import numpy as np
import pickle
import datetime
import timeit
from collections import Counter
from copy import deepcopy
from math import floor,ceil
from scipy.stats import gaussian_kde
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_samples,silhouette_score
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score
import string

In [3]:
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout, Embedding, LSTM, MaxPooling1D, Flatten
from keras.layers import Convolution1D, AveragePooling1D, Conv1D, Input, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Nadam, RMSprop
from keras.utils.np_utils import to_categorical
from keras.models import load_model

Using TensorFlow backend.


In [0]:
from keras.models import Model
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dropout, Embedding, LSTM, MaxPooling1D, Flatten
from keras.layers import Convolution1D, AveragePooling1D, Conv1D, Input, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Nadam, RMSprop
from keras.utils.np_utils import to_categorical
from keras.models import load_model

In [0]:
from scipy.stats import gaussian_kde
from statsmodels.nonparametric.kde import KDEUnivariate
from sklearn.manifold import TSNE
import colorsys

In [0]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import matplotlib.gridspec as gridspec
import nltk

In [14]:
!pip install pyLDAvis

    100% |████████████████████████████████| 1.6MB 15.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
%matplotlib inline

In [0]:
import gensim

In [12]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [0]:
def clean_up(rec):
  table = str.maketrans({key: None for key in string.punctuation})
  rec = str(rec)
  rec = rec.replace('\\n','').replace('\\r','')
  rec = re.sub(r'[^a-zA-Z0-9\.,;?<>/ ]',r'',rec)
  #rec = re.sub(r'\w*\d\w*', '', rec)
  rec = rec.strip()
  rec = re.sub(' +', ' ',rec)
  rec = rec.translate(table)
  rec = rec.lower()
  return rec

In [0]:
import os
drive_path = 'drive/DL_COLAB/'

TICKET_ANALYTICS_PATH = os.path.join(drive_path,"text_analytics","ticket_analysis")

def load_ticket_data(filename, file_path=TICKET_ANALYTICS_PATH):
    csv_path = os.path.join(file_path, filename)
    return pd.read_csv(csv_path,sep=",")

In [0]:
ticket_data_full = load_ticket_data("Telco_Apr-2016.csv",TICKET_ANALYTICS_PATH)

In [0]:
FIELDS = ["Defect ID","Title","Problem Details","Resolution Details","Root Cause"]

In [0]:
ticket_data = ticket_data_full[FIELDS]

In [0]:
problem_desc_list = ticket_data[['Problem Details']].values.tolist()

In [0]:
problem_desc_cleanned = [clean_up(item) for item in problem_desc_list]

In [24]:
problem_desc_cleanned[1:10]

['reason and sub reason section not updated under auc004 and auc005 in dbrd v007',
 'no functional requirements for auc003 dealer changes number of bulk receivers ba will add frs related to auc003',
 'no functional requirements for auc005 dealer places upgrade order ba added frs covering auc005',
 'should be automatically defaulting to all included when it is just going to general market right now',
 'auc covering migration of properties to bulk is missing there are two functional requirements present but no auc ba listed them under deployment option after several discussions ba was convinced to add auc for that purpose',
 'no auc nor functional requirements separation for fulfillment and nonfulfillment dealer activations ba agreed to add auc for nonfulfillment and fulfillment dealer receiver activation dws is uded to activate nonfulfillment fstp riocc and ivr are used for fulfillment dealer activations',
 'confirmation letter codes not updated in cor v200',
 'when attempting to upgrad

In [0]:
problem_desc_cleanned_df = pd.DataFrame(problem_desc_cleanned,columns=['Problem Details'])

In [0]:
complete_text = ''.join(problem_desc_cleanned)

In [29]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
words = nltk.tokenize.word_tokenize(complete_text)

In [0]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
clean_word = [word for word in words if word not in stop]

In [0]:
bigram = gensim.models.Phrases(clean_word, min_count=4, threshold=100) 

In [42]:
trigram = gensim.models.Phrases(bigram[clean_word], threshold=100)

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [0]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [46]:
# Remove Stop Words
data_words_nostops = remove_stopwords(clean_word)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['step']]


In [47]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1)]]


In [48]:
id2word[0]

'step'

In [0]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [56]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.065*"group" + 0.057*"mini" + 0.034*"would" + 0.032*"datum" + '
  '0.031*"attach" + 0.030*"receiver" + 0.027*"new" + 0.027*"response" + '
  '0.025*"bulk" + 0.021*"status"'),
 (1,
  '0.106*"account" + 0.100*"offer" + 0.077*"genie" + 0.039*"display" + '
  '0.038*"customer" + 0.029*"expect" + 0.026*"value" + 0.022*"request" + '
  '0.021*"code" + 0.021*"intent"'),
 (2,
  '0.052*"call" + 0.044*"result" + 0.035*"information" + 0.030*"actual" + '
  '0.027*"telcocompany" + 0.027*"add" + 0.026*"get" + 0.026*"hardware" + '
  '0.024*"equipment" + 0.019*"additional"'),
 (3,
  '0.065*"select" + 0.048*"tab" + 0.044*"service" + 0.043*"issue" + '
  '0.032*"upgrade" + 0.028*"rio" + 0.027*"submit" + 0.019*"dws" + 0.019*"bill" '
  '+ 0.018*"base"'),
 (4,
  '0.106*"order" + 0.076*"click" + 0.049*"test" + 0.035*"exist" + '
  '0.026*"search" + 0.025*"login" + 0.022*"active" + 0.022*"success" + '
  '0.020*"reproduce" + 0.020*"detail"')]


In [57]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      24.409779        1       1 -0.323122 -0.072471
0      19.119654        1       2  0.157579 -0.281993
2      18.962990        1       3  0.042285  0.077990
4      18.844648        1       4  0.049247  0.110677
3      18.662926        1       5  0.074011  0.165798, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
68    Default  1149.000000       account  1149.000000  30.0000  30.0000
61    Default  1078.000000         offer  1078.000000  29.0000  29.0000
22    Default   879.000000         order   879.000000  28.0000  28.0000
383   Default   835.000000         genie   835.000000  27.0000  27.0000
173   Default   630.000000         click   630.000000  26.0000  26.0000
88    Default   549.000000         group   549.000000  25.0000  25.0000
154   Default   533.000000        select   533.000000  24.0000  24.0000
563   Default   483.000000          mini   483.000000  23.0000  23.0000
330   Default   437.000000          call   437.000000  22.0000  22.0000
123   Default   411.000000          test   411.000000  21.0000  21.0000
157   Default   395.000000           tab   395.000000  20.0000  20.0000
174   Default   367.000000        result   367.000000  19.0000  19.0000
269   Default   363.000000       service   363.000000  18.0000  18.0000
158   Default   353.000000         issue   353.000000  17.0000  17.0000
518   Default   421.000000       display   421.000000  16.0000  16.0000
55    Default   409.000000      customer   409.000000  15.0000  15.0000
390   Default   294.000000   information   294.000000  14.0000  14.0000
352   Default   293.000000         exist   293.000000  13.0000  13.0000
76    Default   290.000000         would   290.000000  12.0000  12.0000
21    Default   268.000000       upgrade   268.000000  11.0000  11.0000
219   Default   267.000000         datum   267.000000  10.0000  10.0000
92    Default   311.000000        expect   311.000000   9.0000   9.0000
103   Default   264.000000        attach   264.000000   8.0000   8.0000
15    Default   253.000000      receiver   253.000000   7.0000   7.0000
177   Default   249.000000        actual   249.000000   6.0000   6.0000
91    Default   281.000000         value   281.000000   5.0000   5.0000
161   Default   228.000000           rio   228.000000   4.0000   4.0000
364   Default   227.000000  telcocompany   227.000000   3.0000   3.0000
17    Default   227.000000           add   227.000000   2.0000   2.0000
505   Default   224.000000        submit   224.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
154    Topic5   532.407471        select   533.193359   1.6772  -2.7404
157    Topic5   394.452606           tab   395.249664   1.6766  -3.0403
269    Topic5   363.002258       service   363.790771   1.6765  -3.1234
158    Topic5   352.428711         issue   353.246735   1.6763  -3.1530
21     Topic5   268.048462       upgrade   268.837921   1.6757  -3.4267
161    Topic5   228.111496           rio   228.893204   1.6752  -3.5880
505    Topic5   224.137344        submit   224.930359   1.6751  -3.6056
46     Topic5   159.609024           dws   160.397964   1.6737  -3.9451
164    Topic5   153.425583          bill   154.210999   1.6735  -3.9846
406    Topic5   149.869461          name   150.649643   1.6734  -4.0081
448    Topic5   151.913437          base   152.706345   1.6734  -3.9945
136    Topic5   145.806168          type   146.595261   1.6732  -4.0356
6      Topic5   143.257675       section   144.036407   1.6732  -4.0532
48     Topic5   142.753632      activate   143.542114   1.6731  -4.0567
1508   Topic5   129.170242       loyalty   129.965134   1.6725  -4.1567
852    Topic5   121.139877       session   121.920197   1.6722  -4.2209
201    Topic5   124.151825            tv   124.9773